In [2]:
!pip install pyarrow
!pip install wrds

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 19.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
INFO: pip is looking at multiple versions of wrds to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.13
    Uninstalling SQLAlchemy-2.0.13:
      Successfully uninstalled SQLAlchemy-2.0.13

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import sys
import datetime as dt
import os
import pickle
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import numpy as np
import pandas as pd
import os
import sys
import pyarrow.parquet as pq
import re
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import wrds
import pytz
import math
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize

In [4]:
folder_path = r'/data/workspace_files/parquet_files'
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [5]:
files

['New file']

In [6]:
import numpy as np
import ast

def jaccard_similarity(str1, str2):
    try:
        dict1 = ast.literal_eval(str1)
        dict2 = ast.literal_eval(str2)
        s1 = set(dict1.keys())
        s2 = set(dict2.keys())
        return len(s1.intersection(s2)) / len(s1.union(s2))
    except:
        return -1

def cosine_similarity(str1, str2):
    try:
        dict1 = ast.literal_eval(str1)
        dict2 = ast.literal_eval(str2)
        list1 = [dict1.get(key, 0) for key in set(dict1.keys()) | set(dict2.keys())]
        list2 = [dict2.get(key, 0) for key in set(dict1.keys()) | set(dict2.keys())]
        a = np.array(list1)
        b = np.array(list2)
        cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
        return cos_sim
    except:
        return -1

In [7]:
import pyarrow.parquet as pq

# Specify the path to the Parquet file
parquet_file_path = f'{folder_path}/file_2015.parquet'

# Read the Parquet file into a PyArrow Table
table = pq.read_table(parquet_file_path)

# Convert the PyArrow Table to a Pandas DataFrame (optional)
df = table.to_pandas()

# Now, you can work with the PyArrow Table or Pandas DataFrame as needed
df.head()

FileNotFoundError: FileNotFoundError: /data/workspace_files/parquet_files/file_2015.parquet

In [8]:
import pyarrow.parquet as pq
import pandas as pd

def process_and_save(year):
    folder_path = r'/data/workspace_files/parquet_files'
    curr_file_path = f'{folder_path}/file_{str(year)}.parquet'
    prev_file_path = f'{folder_path}/file_{str(year-1)}.parquet'
    
    # Read only relevant columns from current Parquet file
    table_curr = pq.read_table(curr_file_path, columns=['cik', 'form_type', 'filing_date', 'conformed_period_report', 'doc_dict_text'])
    curr_df = table_curr.to_pandas()
    curr_df['conformed_period_report'] = pd.to_datetime(curr_df['conformed_period_report'])
    curr_df['month'] = curr_df['conformed_period_report'].dt.month
    
    # Read only relevant columns from previous Parquet file
    table_prev = pq.read_table(prev_file_path, columns=['cik', 'form_type', 'conformed_period_report', 'doc_dict_text'])
    prev_df = table_prev.to_pandas()
    prev_df['conformed_period_report'] = pd.to_datetime(prev_df['conformed_period_report'])
    prev_df['month'] = prev_df['conformed_period_report'].dt.month
    
    # Merge DataFrames on 'cik', 'form_type', and 'month'
    df = pd.merge(curr_df, prev_df, on=['cik', 'form_type', 'month'], suffixes=['', '_prev'])
    
    df['cosine_similarity'] = df.apply(lambda x: cosine_similarity(x['doc_dict_text'], x['doc_dict_text_prev']), axis=1)
    df['jaccard_similarity'] = df.apply(lambda x: jaccard_similarity(x['doc_dict_text'], x['doc_dict_text_prev']), axis=1)
    
    # Select and save only necessary columns
    df = df[['cik', 'filing_date', 'cosine_similarity', 'jaccard_similarity']]
    df.to_parquet(f'/data/workspace_files/parquet_files/similarity_scores/by_conformed_period/{str(year)}.parquet')
    # Release memory
    del curr_df, prev_df, table_curr, table_prev, df


dfs = []


In [7]:
for year in range(2015, 2022):
    process_and_save(year)
    print(f'Finished: {year}')

"""final_df = pd.concat(dfs)
print(len(final_df))
print(len(final_df['cik'].unique()))

final_df['filing_monthyr'] = final_df['filing_date'].dt.strftime('%b%Y')"""

Finished: 2000
Finished: 2001
Finished: 2002
Finished: 2003
Finished: 2004
Finished: 2005
Finished: 2006
Finished: 2007
Finished: 2008
Finished: 2009
Finished: 2010
Finished: 2011
Finished: 2012
Finished: 2013
Finished: 2014


<ipython-input-5-580382473648>:22: RuntimeWarning: invalid value encountered in scalar divide
  cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
<ipython-input-5-580382473648>:22: RuntimeWarning: invalid value encountered in scalar divide
  cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


"final_df = pd.concat(dfs)\nprint(len(final_df))\nprint(len(final_df['cik'].unique()))\n\nfinal_df['filing_date'] = pd.to_datetime(final_df['filing_date'])\nfinal_df['filing_monthyr'] = final_df['filing_date'].dt.strftime('%b%Y')"

In [3]:
dfs = []
for yr in range(1996,2019):
    df = pq.read_table(f'/data/workspace_files/parquet_files/similarity_scores/by_conformed_period/{str(yr)}.parquet').to_pandas()
    dfs.append(df)

final_df = pd.concat(dfs)
print(len(final_df))
print(len(final_df['cik'].unique()))

final_df['filing_date'] = pd.to_datetime(final_df['filing_date'])
final_df['filing_monthyr'] = final_df['filing_date'].dt.strftime('%b%Y')

624190
25544


In [10]:
final_df

,cik,filing_date,cosine_similarity,jaccard_similarity,filing_monthyr
0,25354,2007-06-21,0.770233,0.179398,Jun2007
1,102710,1996-07-11,0.016747,0.018256,Jul1996
2,104207,1996-07-11,0.989288,0.614891,Jul1996
3,318251,1996-07-11,0.969808,0.735027,Jul1996
4,34136,1996-07-11,0.985219,0.752542,Jul1996
...,...,...,...,...,...
29501,1082733,2018-04-05,0.996582,0.919569,Apr2018
29502,1041588,2018-06-25,1.000000,1.000000,Jun2018
29503,1041588,2018-06-25,0.993256,0.931889,Jun2018
29504,1041588,2018-06-25,1.000000,1.000000,Jun2018


In [4]:

month_yrs = final_df['filing_monthyr'].unique()
for month_yr in month_yrs:
    df_filt = final_df[final_df['filing_monthyr']==month_yr]
    df_filt.to_parquet(f'/data/workspace_files/parquet_files/similarity_scores/{month_yr}.parquet')

In [9]:
### log-in using your credentials
if 'conn' in locals():
    print("WRDS connection already open!")
else:
    conn = wrds.Connection()

Enter your WRDS username [datalore]: anjali167
Enter your password: ················
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?:  y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [12]:
def get_sd_and_ed_for_returns_old(date, lead):
    start_current_month = date.replace(day=1)
    start_current_month = start_current_month + relativedelta(months = lead)
    start_next_month = start_current_month + relativedelta(months=1)
    #start_current_month = start_current_month + relativedelta(months=-2)

    start_current_month_str = start_current_month.strftime("%Y-%m-%d")
    start_next_month_str = start_next_month.strftime("%Y-%m-%d")

    return start_current_month_str, start_next_month_str

In [13]:
def get_req_tick_cik_map(start_date, end_date):
    # Assuming you have already imported the necessary libraries and connected to your database
    # Assuming you have already imported the necessary libraries and connected to your database

    print(start_date)
    print(end_date)

    sql_query = f"""
    SELECT distinct cik, tic, cusip, cshtrm, trt1m
    FROM comp_na_daily_all.secm
    WHERE datadate >= '{start_date}' and datadate <= '{end_date}'
    """

    cik_tick_map = conn.raw_sql(sql_query)
    cik_tick_map = cik_tick_map.dropna(subset=['cik', 'trt1m'])
    cik_tick_map['cik'] = cik_tick_map['cik'].astype(int)

    print("Total rows:", len(cik_tick_map))
    print("Unique cusip count:", cik_tick_map['cusip'].nunique())

    # Identify 'cik' values with multiple rows
    cik_counts = cik_tick_map['cik'].value_counts()
    multiple_cik_values = cik_counts[cik_counts > 1].index.tolist()

    # Create a mask to filter rows for 'cik' values with multiple rows
    multiple_cik_mask = cik_tick_map['cik'].isin(multiple_cik_values)

    # Create a mask to identify rows with null 'cshtrm'
    null_cshtrm_mask = cik_tick_map['cshtrm'].isnull()

    # Apply both masks to filter rows for 'cik' values with multiple rows
    filtered_cik_tick_map = cik_tick_map[multiple_cik_mask & ~null_cshtrm_mask]

    # Group by 'cik' and apply idxmax() to find the index with the largest 'cshtrm'
    grouped_map = filtered_cik_tick_map.groupby(['cik'])['cshtrm'].idxmax()

    # Use loc to select the rows with the largest 'cshtrm' for 'cik' values with multiple rows
    cik_tick_map_with_max_cshtrm = filtered_cik_tick_map.loc[grouped_map]


    # Select rows for 'cik' values that have only one row
    single_row_cik_values = cik_counts[cik_counts == 1].index.tolist()
    single_row_cik_mask = cik_tick_map['cik'].isin(single_row_cik_values)
    cik_tick_map_single_rows = cik_tick_map[single_row_cik_mask]

    # Append rows for 'cik' values with one row to the previously selected rows
    final_cik_tick_map = pd.concat([cik_tick_map_with_max_cshtrm, cik_tick_map_single_rows], axis = 0)
    # Rename the 'trt1m' column to 'ret'
    final_cik_tick_map.rename(columns={'trt1m': 'ret'}, inplace=True)
    
    print("Total rows after selecting rows with max cshtrm:", len(final_cik_tick_map))


    return final_cik_tick_map

In [ ]:
def create_directories():
    folder_path = '/data/workspace_files/parquet_files'
    for score_type in ['cosine_similarity', 'jaccard_similarity']:
            for lead in range(1,13):
                folder_path = f'{folder_path}/portfolio_lead_lag/{score_type}/{lead}
                print(folder_path)
                os.makedirs(folder_path, exist_ok = True)

In [46]:
month_yr = 'Jan1996'
date_obj = datetime.strptime(month_yr, '%b%Y')
sd_for_cik_ticker_map, ed_for_cik_ticker_map = get_sd_and_ed_for_returns_old(date_obj)
print(sd_for_cik_ticker_map, ed_for_cik_ticker_map)
cik_tick_map_req = get_req_tick_cik_map(start_date=sd_for_cik_ticker_map, end_date=ed_for_cik_ticker_map)
score_type = 'jaccard_similarity'
score_path = f'{folder_path}/similarity_scores/{month_yr}.parquet'
scores = pq.read_table(score_path).to_pandas()
cik_tick_map_req['ret_clipped'] = cik_tick_map_req['ret'].clip(lower=-50.0, upper=50.0)
scores = pd.merge(scores, cik_tick_map_req, how = 'inner', left_on = ['cik'], right_on = ['cik'])
print(len(scores))

# Extract the similarity scores
similarity_scores = scores[score_type]

# Create buckets (quantiles) for long and short positions
num_buckets = 5  # Number of quantiles, you can adjust this
buckets = pd.qcut(similarity_scores, q=num_buckets, labels=False, duplicates='drop')

# Assign long and short positions based on buckets
long_positions = scores[buckets == (num_buckets - 1)]
short_positions = scores[buckets == 0]
long_positions['pos_type'] = 1
short_positions['pos_type'] = -1
long_positions['weight'] = long_positions['cshtrm'] / long_positions['cshtrm'].sum()
short_positions['weight'] = -1*short_positions['cshtrm'] / short_positions['cshtrm'].sum()


positions = pd.concat([long_positions, short_positions])
positions['num_months'] = 1
positions['date'] = month_yr
positions = positions[['date', 'cik', 'pos_type', 'num_months', 'weight', 'ret', 'ret_clipped', 'filing_date', 'cshtrm']]
positions.to_parquet(f'/data/workspace_files/parquet_files/portfolios2/{score_type}/{month_yr}.parquet')

print(f'Num Holdings: {len(positions)}')
    

1996-02-01 1996-03-01
1996-02-01
1996-03-01
Total rows: 10368
Unique cusip count: 10176
Total rows after selecting rows with max cshtrm: 9975
2
Num Holdings: 2


<ipython-input-46-01ec7d437913>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_positions['pos_type'] = 1
<ipython-input-46-01ec7d437913>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_positions['pos_type'] = -1
<ipython-input-46-01ec7d437913>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [14]:
import pandas as pd
import pyarrow.parquet as pq

folder_path = '/data/workspace_files/parquet_files'
month_dict = { 1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

# Loop from March 2015 to March 2021
for score_type in ['cosine_similarity', 'jaccard_similarity']:
    for year in range(1996, 2018):
        for month in range(1, 13):
            for lead in range(1,13):
                if year == 1996 and month <= 1:
                    continue
                month_yr = f"{month_dict[month]}{year}"
                # Load similarity scores
                score_path = f'{folder_path}/similarity_scores/{month_yr}.parquet'
                scores = pq.read_table(score_path).to_pandas()
                print(len(scores))
                date_obj = datetime.strptime(month_yr, '%b%Y')
                sd_for_cik_ticker_map, ed_for_cik_ticker_map = get_sd_and_ed_for_returns_old(date_obj, lead)
                cik_tick_map_req = get_req_tick_cik_map(start_date=sd_for_cik_ticker_map, end_date=ed_for_cik_ticker_map)
                cik_tick_map_req['ret_clipped'] = cik_tick_map_req['ret'].clip(lower=-50.0, upper=50.0)
                scores = pd.merge(scores, cik_tick_map_req, how = 'inner', left_on = ['cik'], right_on = ['cik'])
                print(len(scores))

                # Extract similarity scores
                similarity_scores = scores[score_type]

                # Create buckets (quantiles) for long and short positions
                num_buckets = 5  # Number of quantiles, you can adjust this
                buckets = pd.qcut(similarity_scores, q=num_buckets, labels=False, duplicates='drop')

                # Assign long and short positions based on buckets
                long_positions = scores[buckets == (num_buckets - 1)]
                short_positions = scores[buckets == 0]
                long_positions['pos_type'] = 1
                short_positions['pos_type'] = -1
                positions = pd.concat([long_positions, short_positions])
                positions['num_months'] = 1

                # Load previous positions
                if month == 1:
                    prev_month = 12
                    prev_yr = year-1
                else:
                    prev_month = month-1
                    prev_yr = year

                prev_month_yr = f"{month_dict[prev_month]}{prev_yr}"
                port_path = f'{folder_path}/portfolio_lead_lag/{score_type}/{prev_month_yr}.parquet'
                prev_positions = pq.read_table(port_path).to_pandas()
                prev_positions['num_months'] = prev_positions['num_months'] + 1
                prev_positions = prev_positions[prev_positions['num_months'] <= 6]
                print(f'Prev Holdings {len(prev_positions)}')

                # Concatenate current and previous positions
                positions = pd.concat([positions, prev_positions]).drop_duplicates(subset='cik')

                # Separate long and short positions
                long_positions = positions[positions['pos_type'] == 1]
                short_positions = positions[positions['pos_type'] == -1]

                # Calculate weights
                long_positions['weight'] = 1/len(long_positions)
                short_positions['weight'] = -1/len(short_positions)

                # Concatenate long and short positions
                positions = pd.concat([long_positions, short_positions])

                # Assign the date
                positions['date'] = month_yr
                positions = positions[['date', 'cik', 'pos_type', 'num_months', 'weight', 'filing_date']]
                positions = pd.merge(positions, cik_tick_map_req, how = 'inner', left_on = ['cik'], right_on = ['cik'])

                # Save the positions to Parquet file
                positions[['date', 'cik', 'pos_type', 'num_months', 'weight', 'ret', 'ret_clipped', 'filing_date', 'cshtrm']].to_parquet(
                    f'{folder_path}/portfolio_lead_lag/{score_type}/{month_yr}.parquet'
                )
                print(f'Num Holdings: {len(positions)}')

20
1996-03-01
1996-04-01
Total rows: 10416
Unique cusip count: 10217
Total rows after selecting rows with max cshtrm: 10021
18


<ipython-input-14-01f7ad8e37b7>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_positions['pos_type'] = 1
<ipython-input-14-01f7ad8e37b7>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_positions['pos_type'] = -1


FileNotFoundError: FileNotFoundError: /data/workspace_files/parquet_files/portfolios/cosine_similarity/Jan1996.parquet

In [44]:
positions

,date,cik,pos_type,num_months,weight,ret,ret_clipped,filing_date,cshtrm
0,Jan1996,48465,1,1,1.0,3.0769,3.0769,1996-01-26,1090700.0
1,Jan1996,107681,-1,1,-1.0,-2.2776,-2.2776,1996-01-29,2544000.0


In [16]:
positions

,date,cik,pos_type,num_months,weight,filing_date,ret,ret_clipped
0,Dec2019,1069533,1,1,0.001091,2019-12-03,-10.7943,-10.7943
1,Dec2019,924717,1,1,0.001091,2019-12-03,-4.9240,-4.9240
2,Dec2019,1530425,1,1,0.001091,2019-12-03,0.0000,0.0000
3,Dec2019,1687926,1,1,0.001091,2019-12-05,3.0928,3.0928
4,Dec2019,876523,1,1,0.001091,2019-12-05,-8.7977,-8.7977
...,...,...,...,...,...,...,...,...
1829,Dec2019,895419,-1,3,-0.001082,2019-10-31,0.7367,0.7367
1830,Dec2019,919956,-1,3,-0.001082,2019-10-31,3.6364,3.6364
1831,Dec2019,944314,-1,3,-0.001082,2019-10-31,5.5135,5.5135
1832,Dec2019,944695,-1,3,-0.001082,2019-10-31,1.3975,1.3975


In [13]:
scores

,cik,filing_date,cosine_similarity,jaccard_similarity,filing_monthyr,ret,ret_clipped
0,773717,2019-12-06,0.985736,0.889655,Dec2019,-5.9508,-5.9508
1,773717,2019-12-23,0.983507,0.881145,Dec2019,-5.9508,-5.9508
2,1067873,2019-12-16,0.996564,0.747925,Dec2019,-48.6116,-48.6116
3,1067873,2019-12-20,0.996909,0.764706,Dec2019,-48.6116,-48.6116
4,1067873,2019-12-23,0.996581,0.804622,Dec2019,-48.6116,-48.6116
...,...,...,...,...,...,...,...
363,1625376,2019-12-30,0.982843,0.695219,Dec2019,0.0000,0.0000
364,1681282,2019-12-30,0.976105,0.686209,Dec2019,-78.0000,-50.0000
365,1310630,2019-12-31,0.995134,0.882548,Dec2019,0.0000,0.0000
366,1502152,2019-12-31,0.976843,0.725932,Dec2019,40.3226,40.3226


In [4]:
import pandas as pd
import pyarrow.parquet as pq

score_type = 'cosine_similarity'

folder_path = '/data/workspace_files/parquet_files'
month_dict = { 1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

# Loop from March 2015 to March 2021
for year in range(1997, 2020):
    for month in range(1, 13):
        month_yr = f'{month_dict[month]}{str(year)}'
        long_positions = pq.read_table(f'{folder_path}/portfolios/{score_type}/Quantile_5_{month_yr}.parquet').to_pandas()
        long_positions['pos_type'] = 1
        short_positions = pq.read_table(f'{folder_path}/portfolios/{score_type}/Quantile_1_{month_yr}.parquet').to_pandas()
        short_positions['pos_type'] = -1
        short_positions['weight'] = -1*short_positions['weight']
        positions = pd.concat([long_positions, short_positions])
        positions.to_parquet(f'{folder_path}/portfolios/{month_yr}_v2.parquet')




In [24]:
def get_req_tick_cik_map(start_date, end_date):
    # Assuming you have already imported the necessary libraries and connected to your database
    # Assuming you have already imported the necessary libraries and connected to your database

    print(start_date)
    print(end_date)

    sql_query = f"""
    SELECT *
    FROM crsp_a_stock.msf
    WHERE ticker = 'AAPL'
    """

    cik_tick_map = conn.raw_sql(sql_query)
    return cik_tick_map
    

In [34]:
mcap, ret = get_req_tick_cik_map('2010-01-01', '2010-02-01')

2010-01-01
2010-02-01


ProgrammingError: ProgrammingError: (psycopg2.errors.UndefinedColumn) column "ipermno" does not exist
LINE 2:     SELECT distinct cik, tic, cusip, cshtrm, trt1m, ipermno
                                                            ^

[SQL: 
    SELECT distinct cik, tic, cusip, cshtrm, trt1m, ipermno
    FROM comp_na_daily_all.secm
    WHERE datadate >= '2010-01-01' and datadate <= '2010-02-01'
    ]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [17]:
mcap.head()

,cusip,permno,prc,shrout,cfacpr,cfacshr,Mcap
0,61745P86,86485.0,13.430000,17484.0,1.00,1.00,2.348101e+05
1,11133T10,91849.0,21.719999,136863.0,1.00,1.00,2.972664e+06
2,29273V10,91111.0,32.029999,222898.0,4.00,4.00,7.139423e+06
3,00211E10,93197.0,-2.530000,22770.0,1.00,1.00,-5.760810e+04
4,74347X17,92963.0,29.480000,150.0,0.25,0.25,4.422000e+03


In [20]:
start_date = '2010-01-01'
end_date = '2010-02-01'
sql_query = f"""
        SELECT *
        FROM comp_na_daily_all.secm
        LIMIT 10
    """
mcap = conn.raw_sql(sql_query)

In [29]:
ret[ret['tic']=='MSFT']

,cik,tic,cusip,cshtrm,ret
6466,789019,MSFT,594918104,1.357981e+09,-7.5459


In [32]:
mcap[mcap['permno']==10107]

,cusip,permno,prc,shrout,cfacpr,cfacshr,Mcap
2602,59491810,10107.0,28.18,8770461.0,1.0,1.0,2.471516e+08
